In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import random
import numpy as np
from collections import defaultdict
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms as T, datasets, models
from tqdm import tqdm

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [19]:
num_classes = 65  
batch_size = 64
source_epochs = 5 
adapt_epochs = 5
learning_rate = 1e-3
momentum = 0.9

In [20]:
source_transform = T.Compose([
    T.Resize(256),
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

weak_transform = T.Compose([
    T.Resize(256),
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
strong_transform = T.Compose([
    T.Resize(256),
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    T.RandomGrayscale(p=0.2),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [21]:
class OfficeHomeTargetDataset(Dataset):
    def __init__(self, root, transform_weak, transform_strong=None):
        self.dataset = datasets.ImageFolder(root, transform=None)
        self.transform_weak = transform_weak
        self.transform_strong = transform_strong if transform_strong is not None else transform_weak

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        img_w = self.transform_weak(img)
        img_s = self.transform_strong(img)
        return img_w, img_s, label


In [ ]:
class SFUDAModel(nn.Module):
    def __init__(self, num_classes):
        super(SFUDAModel, self).__init__()
        backbone = models.resnet50(pretrained=True)
        # backbone = models.resnet101(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-1])
        self.fc = nn.Linear(backbone.fc.in_features, num_classes)
        
    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        logits = self.fc(features)
        return features, logits

In [23]:
class MemoryBank:
    def __init__(self, feature_dim, dataset_size, momentum=0.2):
        self.feature_dim = feature_dim
        self.momentum = momentum
        self.bank = torch.zeros(dataset_size, feature_dim).to(device)
    
    def update(self, indices, features):
        with torch.no_grad():
            self.bank[indices] = self.momentum * self.bank[indices] + (1 - self.momentum) * features
            self.bank[indices] = F.normalize(self.bank[indices], dim=1)
    
    def get_features(self):
        return self.bank

In [24]:
def self_training_loss(p_w, p_s, pseudo_label, num_classes, lambda_diversity=0.1, omega_entropy=0.1):
    ce_loss = F.cross_entropy(p_w, pseudo_label.long()) + F.cross_entropy(p_s, pseudo_label.long())
    mean_pred = p_w.mean(dim=0)
    diversity_loss = F.kl_div(mean_pred.log(), torch.full_like(mean_pred, 1/num_classes), reduction='batchmean')
    entropy_loss = - (p_w * p_w.log()).sum(dim=1).mean()
    return ce_loss + lambda_diversity * diversity_loss + omega_entropy * entropy_loss

def contrastive_loss(anchor, positive, negatives, temperature=0.05):
    anchor = F.normalize(anchor, dim=1)
    positive = F.normalize(positive, dim=1)
    negatives = F.normalize(negatives, dim=1)
    
    pos_sim = torch.sum(anchor * positive, dim=1) / temperature
    neg_sim = torch.matmul(anchor, negatives.t()) / temperature
    logits = torch.cat([pos_sim.unsqueeze(1), neg_sim], dim=1)
    labels = torch.zeros(anchor.size(0), dtype=torch.long).to(device)
    loss = F.cross_entropy(logits, labels)
    return loss

def mmd_loss(source_features, target_features):
    mean_source = source_features.mean(dim=0)
    mean_target = target_features.mean(dim=0)
    loss = torch.norm(mean_source - mean_target, p=2)**2
    return loss

In [25]:
def compute_centroids(features, pseudo_labels, num_classes):
    centroids = []
    for c in range(num_classes):
        mask = (pseudo_labels == c)
        if mask.sum() == 0:
            centroid = torch.zeros(features.size(1)).to(device)
        else:
            centroid = features[mask].mean(dim=0)
            centroid = F.normalize(centroid.unsqueeze(0), dim=1).squeeze(0)
        centroids.append(centroid)
    centroids = torch.stack(centroids, dim=0)
    return centroids

def assign_pseudo_labels(features, centroids):
    features = F.normalize(features, dim=1)
    sim = torch.matmul(features, centroids.t())
    pseudo_labels = sim.argmax(dim=1)
    confidence, _ = sim.max(dim=1)
    return pseudo_labels, confidence

def divide_samples(confidence, threshold=0.8):
    source_like_idx = (confidence >= threshold).nonzero(as_tuple=False).squeeze()
    target_specific_idx = (confidence < threshold).nonzero(as_tuple=False).squeeze()
    return source_like_idx, target_specific_idx

def update_pseudo_labels(model, dataloader, tau=0.8):
    model.eval()
    all_features = []
    all_indices = []
    with torch.no_grad():
        for batch_idx, (img_w, _, _) in enumerate(dataloader):
            img_w = img_w.to(device)
            features, _ = model(img_w)
            features = F.normalize(features, dim=1)
            all_features.append(features)
            indices = torch.arange(batch_idx * batch_size, min((batch_idx+1)*batch_size, len(dataloader.dataset))).to(device)
            all_indices.append(indices)
    all_features = torch.cat(all_features, dim=0)
    all_indices = torch.cat(all_indices, dim=0)
    pseudo_labels = all_features.argmax(dim=1) 
    centroids = compute_centroids(all_features, pseudo_labels, num_classes)
    pseudo_labels, confidence = assign_pseudo_labels(all_features, centroids)
    source_like_idx, target_specific_idx = divide_samples(confidence, threshold=tau)
    model.train()
    return all_indices, pseudo_labels, confidence, source_like_idx, target_specific_idx, centroids



In [26]:
if __name__ == "__main__":
    art_data_dir = "../data/Office-Home/Art"
    real_data_dir = "../data/Office-Home/Real World"
    
    full_art_dataset = OfficeHomeTargetDataset(art_data_dir, transform_weak=source_transform, transform_strong=source_transform)
    art_subset_indices = list(range(min(1000, len(full_art_dataset))))
    art_subset = Subset(full_art_dataset, art_subset_indices)
    art_loader = DataLoader(art_subset, batch_size=batch_size, shuffle=True, num_workers=0)
    
    source_model = SFUDAModel(num_classes=num_classes).to(device)
    optimizer_source = optim.SGD(source_model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=5e-4)
    
    print("Starting source training on Art subset...")
    for epoch in range(source_epochs):
        running_loss = 0.0
        for img, _, labels in tqdm(art_loader, desc=f"Source Epoch {epoch+1}/{source_epochs}", ncols=80):
            img = img.to(device)
            labels = labels.to(device)
            features, logits = source_model(img)
            loss = F.cross_entropy(logits, labels)
            optimizer_source.zero_grad()
            loss.backward()
            optimizer_source.step()
            running_loss += loss.item()
        print(f"Source Epoch [{epoch+1}/{source_epochs}], Loss: {running_loss/len(art_loader):.4f}")
    
    
    torch.save(source_model.state_dict(), "../models/source_model_art.pth")
    print("Source training finished and model saved.")


/home/charanganeshcharanganesh/miniconda3/envs/AI/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/charanganeshcharanganesh/miniconda3/envs/AI/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Starting source training on Art subset...


Source Epoch 1/5: 100%|█████████████████████████| 16/16 [10:10<00:00, 38.14s/it]


Source Epoch [1/5], Loss: 3.7803


Source Epoch 2/5: 100%|█████████████████████████| 16/16 [11:03<00:00, 41.44s/it]


...

Source Epoch [3/5], Loss: 2.0236


Source Epoch 4/5: 100%|█████████████████████████| 16/16 [01:03<00:00,  4.00s/it]


Source Epoch [4/5], Loss: 1.6440


Source Epoch 5/5: 100%|█████████████████████████| 16/16 [01:05<00:00,  4.06s/it]


Source Epoch [5/5], Loss: 1.3756
Source training finished and model saved.


In [27]:
if __name__ == "__main__":
    adapted_model = SFUDAModel(num_classes=num_classes).to(device)
    adapted_model.load_state_dict(torch.load("../models/source_model_art.pth"))
    
    for param in adapted_model.fc.parameters():
        param.requires_grad = False
    full_real_dataset = OfficeHomeTargetDataset(real_data_dir, transform_weak=weak_transform, transform_strong=strong_transform)
    real_subset_indices = list(range(min(500, len(full_real_dataset))))
    real_subset = Subset(full_real_dataset, real_subset_indices)
    real_loader = DataLoader(real_subset, batch_size=batch_size, shuffle=True, num_workers=0)
    

    dataset_size = len(real_subset)
    feature_dim = 2048
    memory_bank = MemoryBank(feature_dim, dataset_size, momentum=0.2)
    
    optimizer_adapt = optim.SGD(adapted_model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=5e-4)
    alpha = 1.0  
    beta = 0.5   
    gamma = 1.0 
    dataset_indices = np.arange(dataset_size)
    
    print("Starting adaptation on Real World subset...")
    for epoch in range(adapt_epochs):
        all_indices, pseudo_labels, confidence, src_like_idx, tgt_spec_idx, centroids = update_pseudo_labels(adapted_model, real_loader, tau=0.8)
        running_loss = 0.0
        for batch_idx, (img_w, img_s, _) in enumerate(tqdm(real_loader, desc=f"Adapt Epoch {epoch+1}/{adapt_epochs}", ncols=80)):
            batch_start = batch_idx * batch_size
            batch_end = batch_start + img_w.size(0)
            batch_indices = torch.tensor(dataset_indices[batch_start:batch_end]).to(device)
            
            img_w = img_w.to(device)
            img_s = img_s.to(device)
            
            features_w, logits_w = adapted_model(img_w)
            features_s, logits_s = adapted_model(img_s)
            p_w = F.softmax(logits_w, dim=1)
            p_s = F.softmax(logits_s, dim=1)
            batch_pseudo = pseudo_labels[batch_indices]
            
            loss_self = self_training_loss(p_w, p_s, batch_pseudo, num_classes)
            batch_features = F.normalize(features_w, dim=1)
            positive_proto = centroids[batch_pseudo]
            bank_features = memory_bank.get_features()
            
            loss_contrastive = 0.0
            for i in range(batch_features.size(0)):
                neg_mask = (pseudo_labels != batch_pseudo[i])
                negatives = bank_features[neg_mask]
                if negatives.size(0) > 0:
                    loss_contrastive += contrastive_loss(batch_features[i].unsqueeze(0),
                                                         positive_proto[i].unsqueeze(0),
                                                         negatives)
            loss_contrastive = loss_contrastive / batch_features.size(0)
            
            src_mask = torch.tensor([idx.item() in src_like_idx.cpu().numpy() for idx in batch_indices]).bool()
            tgt_mask = torch.tensor([idx.item() in tgt_spec_idx.cpu().numpy() for idx in batch_indices]).bool()
            if src_mask.sum() > 0 and tgt_mask.sum() > 0:
                src_feats = batch_features[src_mask]
                tgt_feats = batch_features[tgt_mask]
                loss_mmd = mmd_loss(src_feats, tgt_feats)
            else:
                loss_mmd = torch.tensor(0.0).to(device)
            
            loss = alpha * loss_self + gamma * loss_contrastive + beta * loss_mmd
            
            optimizer_adapt.zero_grad()
            loss.backward()
            optimizer_adapt.step()
            
            memory_bank.update(batch_indices, batch_features.detach())
            running_loss += loss.item()
        print(f"Adapt Epoch [{epoch+1}/{adapt_epochs}], Loss: {running_loss/len(real_loader):.4f}")
    
    def evaluate(model, dataloader):
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for img_w, _, labels in dataloader:
                img_w = img_w.to(device)
                labels = labels.to(device)
                _, logits = model(img_w)
                preds = logits.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        return 100.0 * correct / total
    
    accuracy = evaluate(adapted_model, real_loader)
    print(f"Adapted Target Domain Accuracy on Real World Subset: {accuracy:.2f}%")


/home/charanganeshcharanganesh/miniconda3/envs/AI/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/charanganeshcharanganesh/miniconda3/envs/AI/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_8645/128733223.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://gi

Starting adaptation on Real World subset...


Adapt Epoch 1/5: 100%|████████████████████████████| 8/8 [01:21<00:00, 10.20s/it]


Adapt Epoch [1/5], Loss: 14.5052


Adapt Epoch 2/5: 100%|████████████████████████████| 8/8 [01:29<00:00, 11.14s/it]


...

Adapt Epoch [3/5], Loss: 16.1227


Adapt Epoch 4/5: 100%|████████████████████████████| 8/8 [01:51<00:00, 13.90s/it]


Adapt Epoch [4/5], Loss: 15.4506


Adapt Epoch 5/5: 100%|████████████████████████████| 8/8 [01:25<00:00, 10.67s/it]


Adapt Epoch [5/5], Loss: 15.4457
Adapted Target Domain Accuracy on Real World Subset: 54.80%
